<a href="https://colab.research.google.com/github/marcinszulczyk/kaggle_titanic_solution/blob/feature%2FMS_First_Test/kaggle_titanic_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **KAGGLE RANKING**: 33 897

# **KAGGLE SCORE**:   0.55023

# Import bibliotek

In [11]:
import pandas as pd

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# Pobranie danych danych

## Link do Kaggle

https://www.kaggle.com/c/titanic/overview

## Wczytanie danych

In [12]:
df_train = pd.read_csv("https://raw.githubusercontent.com/marcinszulczyk/kaggle_titanic_solution/c48ff96bbfd29c691a78a0e46e3dc2abaeec65c1/data/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/marcinszulczyk/kaggle_titanic_solution/c48ff96bbfd29c691a78a0e46e3dc2abaeec65c1/data/test.csv")

## Podstawowe informacje o danych

In [16]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [15]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [17]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [18]:
df_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [14]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


### Kolumny


---


**PassengerId**  

ID pasażera


---

**Survived**

Czy pasażer został uratowany

0 - nie został uratowany

1 - został uratowany

---

**Pclass**

Klasa jaką podróżował pasażer

1 - Pierwsza klasa (najwyższy standard)

2 - Druga klasa

3 - Trzecia klasa (najniższy standard)

---

**Name** 

Imię pasażera

Imię, nazwisto, tytuł itp.

---

**Sex**        

Płeć pasażera

male - mężczyzna

female - kobieta

---

**Age**          

Wiek pasażera

Jeżeli było dziecko ale nie miało rodziców to oznaczało, że podróżowało z nianią.

---

**SibSp**

Ilość rodzeństwa na pokładzie - bracia, siostry itp.

---

**Parch**

Ilość rodziców na pokładzie

---

**Ticket**

Numer biletu

---

**Fare**

Koszt biletu

---

**Cabin**

Przedział na statu

---

**Embarked**

Port rozpoczęcia podróży

C - Cherbourg

Q - Queenstown

S - Southampton

---




# Model uczenia

## Pierwszy model

Wykonay w celu sprawdzenia poprawności wygenerowania wyniku i wysyłanego pliku na kaggle.

In [22]:
X = df_train[['Pclass']].values
y = df_train['Survived'].values

In [23]:
model = DummyClassifier()
model.fit(X, y)
y_pred = model.predict(X)

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


In [24]:
score = accuracy_score(y, y_pred)
print("score: %.2f" % score)

score: 0.52


In [26]:
X = df_test[['Pclass']].values

df_test['Survived'] = model.predict(X)
df_test[ ['PassengerId', 'Survived'] ].to_csv('dummy.csv', index=False)